### Rival posession overall

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [61]:
#df = pd.read_csv('/Users/pgule/Documents/Sample Data/epl.csv', delimiter=',')
df = pd.read_csv('/Users/luisalawson/Downloads/Sample Data/epl.csv', delimiter=';')

In [148]:
def match_outcome(df):
    relevant_df = df[df['description'].isin(['Goal', 'Pass'])]
    matches = relevant_df['match_id'].unique()
    match_data = []

    for match in matches:
        match_df = relevant_df[relevant_df['match_id'] == match].reset_index(drop=True)
        
        # Extract unique values
        home_team_id = match_df['home_team_id'].iloc[0]
        away_team_id = match_df['away_team_id'].iloc[0]
        home_team_name = match_df['home_team_name'].iloc[0]
        away_team_name = match_df['away_team_name'].iloc[0]

        home_score = 0
        away_score = 0

        for i, row in match_df.iterrows():
            team_id = row['team_id']
            
            if team_id == home_team_id and row['description'] == 'Goal' and row['outcome'] == 1:
              
                if i + 1 < len(match_df):
                    next_row = match_df.iloc[i + 1]
                    
                    if next_row['team_id'] == home_team_id:
                        away_score += 1
                    else:
                        home_score += 1
                else:
                    home_score += 1
            elif team_id == away_team_id and row['description'] == 'Goal' and row['outcome'] == 1:
                # Check if the next event is for the same team
                if i + 1 < len(match_df):
                    next_row = match_df.iloc[i + 1]
                    if next_row['team_id'] == away_team_id:
                        home_score += 1
                    else:
                        away_score += 1
                else:
                    away_score += 1

        match_stats = {
            'match_id': match,
            'home_team': home_team_name,
            'away_team': away_team_name,
            'home_score': home_score,
            'away_score': away_score
        }

        match_data.append(match_stats)

    return pd.DataFrame(match_data)

In [149]:
matches_outcome =match_outcome(df)

In [150]:
excel_file_path = '/Users/luisalawson/Downloads/prueba.xlsx'  

matches_outcome.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")

AttributeError: 'list' object has no attribute 'to_excel'

In [115]:
matches_outcome

,season_id,match_id,home_team_id,home_team_name,away_team_id,away_team_name,id,event_id,date,time,...,sec,team_id,player_id,playerName,playerPosition,x,y,type,description,outcome
4,2012,441999,3,Arsenal,56,Sunderland,164697335,3,18Aug2012,3:02:00 PM,...,1,3,17733.0,"Podolski, Lukas",Striker,50,50,1,Pass,1
5,2012,441999,3,Arsenal,56,Sunderland,763384109,4,18Aug2012,3:02:01 PM,...,2,3,19524.0,"Cazorla, Santiago",Midfielder,"53,7","50,3",1,Pass,1
6,2012,441999,3,Arsenal,56,Sunderland,708867683,5,18Aug2012,3:02:04 PM,...,4,3,15943.0,"Vermaelen, Thomas",Defender,"30,6","52,5",1,Pass,1
10,2012,441999,3,Arsenal,56,Sunderland,861562098,4,18Aug2012,3:02:18 PM,...,18,56,15073.0,"Richardson, Kieran",Midfielder,"22,5",100,1,Pass,0
11,2012,441999,3,Arsenal,56,Sunderland,119149693,8,18Aug2012,3:02:20 PM,...,20,3,80254.0,"Jenkinson, Carl",Defender,"58,9","3,9",1,Pass,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,2012,441999,3,Arsenal,56,Sunderland,445004145,1103,18Aug2012,4:52:56 PM,...,37,3,19524.0,"Cazorla, Santiago",Midfielder,"91,5",0,1,Pass,1
1774,2012,441999,3,Arsenal,56,Sunderland,254257077,1104,18Aug2012,4:52:57 PM,...,39,3,80254.0,"Jenkinson, Carl",Defender,74,"6,7",1,Pass,1
1775,2012,441999,3,Arsenal,56,Sunderland,925234117,1105,18Aug2012,4:52:58 PM,...,40,3,8758.0,"Arteta, Mikel",Midfielder,74,19,1,Pass,1
1776,2012,441999,3,Arsenal,56,Sunderland,1029290489,1106,18Aug2012,4:53:01 PM,...,42,3,19524.0,"Cazorla, Santiago",Midfielder,"94,5","22,3",1,Pass,0


In [95]:
def group_plays(df):
    match_stats_list = []
    goalkeeper_events = ['Attempt saved', 'Save', 'Pass', 'Keeper pick-up']
    unique_matches = df['match_id'].unique()

    for currentmatch in unique_matches:
        match_df = df[df['match_id'] == currentmatch]

        home_team_id = match_df['home_team_id'].iloc[0]
        away_team_id = match_df['away_team_id'].iloc[0]
        home_team_name = match_df['home_team_name'].iloc[0]
        away_team_name = match_df['away_team_name'].iloc[0]

        home_plays = 0
        away_plays = 0
        home_passes = 0
        away_passes = 0
        home_dangerous_play = 0
        away_dangerous_play = 0
        
        current_team = None
        last_event_id = None
        in_play = False
        current_passes = 0
        
        # home_team_first_half_side = (0,50)
        # away_team_first_half_side = (0,50)
        home_team_danger = (70,100)
        away_team_danger = (70,100)

        for i, row in match_df.iterrows():
            team_id = row['team_id']
            event_id = row['event_id']

            if not in_play:
                in_play = True
                current_team = team_id
                last_event_id = event_id
                current_passes = 0
            else:
                if team_id != current_team or event_id != (last_event_id + 1):
                    # see if it is a dangerous play
                    x = float(row['x'].replace(',', '.'))
                    if current_team == home_team_id:
                        home_plays += 1
                        home_passes += current_passes
                        if home_team_danger[0] <= x <= home_team_danger[1]:
                            home_dangerous_play += 1
                    else:
                        away_plays += 1
                        away_passes += current_passes
                        if away_team_danger[0] <= x <= away_team_danger[1]:
                            away_dangerous_play += 1
                    
                    current_team = team_id
                    last_event_id = event_id
                    current_passes = 0
                else:
                    # Continue current play
                    last_event_id = event_id


            # if home_team_first_half_side is None:
            #     x_coordinate = float(row['x'].replace(',', '.'))
            #     if (row['team_id']== home_team_id):
            #         if (row['playerPosition'] == 'Goalkeeper') and (row['period_id'] == 1) and (row['description'] in goalkeeper_events) :
            #             if x_coordinate >50:
            #                 home_team_first_half_side = (50,100)
            #                 home_team_danger = (70,100)
            #             else:
            #                 home_team_first_half_side = (0,50)
            #                 home_team_danger = (0,30)
            
            # if away_team_first_half_side is None:
            #     x_coordinate = float(row['x'].replace(',', '.'))
            #     if (row['team_id']== away_team_id):
            #         if (row['playerPosition'] == 'Goalkeeper') and (row['period_id'] == 1) and (row['description'] in goalkeeper_events) :
            #             if x_coordinate >50:
            #                 away_team_first_half_side = (50,100)
            #                 away_team_danger = (70,100)
            #             else:
            #                 away_team_first_half_side = (0,50)
            #                 away_team_danger = (0,30)

        
            if row['description'] == 'Pass' and row['outcome'] == 1:
                current_passes += 1
        
        
        match_stats = {
            'match_id': currentmatch,
            'home_team': home_team_name,
            'away_team': away_team_name,
            'home_team_plays': home_plays,
            'away_team_plays': away_plays,
            'home_passes_per_play': round(home_passes / home_plays, 4) if home_plays > 0 else 0,
            'away_passes_per_play': round(away_passes / away_plays, 4) if away_plays > 0 else 0,
            'home_faces_dangerous_play': home_dangerous_play,
            'away_faces_dangerous_play': away_dangerous_play,
            # 'home_team_first_half_side': home_team_first_half_side,
            # 'away_team_first_half_side': away_team_first_half_side,
            'home_team_danger': home_team_danger,
            'away_team_danger': away_team_danger,
            'home_RPDA': round(away_dangerous_play/away_plays, 4),
            'away_RPDA': round(home_dangerous_play/home_plays, 4)
        }

        match_stats_list.append(match_stats)
    
    return pd.DataFrame(match_stats_list)



In [96]:
df_grouped_plays = group_plays(df)

In [99]:
df_grouped_plays

,match_id,home_team,away_team,home_score,away_score,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_faces_dangerous_play,away_faces_dangerous_play,home_team_danger,away_team_danger,home_RPDA,away_RPDA
0,441999,Arsenal,Sunderland,0,0,315,322,2.1206,0.7391,28,96,"(70, 100)","(70, 100)",0.2981,0.0889
1,442000,Everton,Manchester United,0,0,350,340,0.6514,1.7794,69,67,"(70, 100)","(70, 100)",0.1971,0.1971
2,442001,Fulham,Norwich City,0,0,287,307,1.8850,1.1107,43,66,"(70, 100)","(70, 100)",0.2150,0.1498
3,442002,Manchester City,Southampton,0,0,300,278,2.2100,1.2266,53,67,"(70, 100)","(70, 100)",0.2410,0.1767
4,442003,Newcastle United,Tottenham Hotspur,0,0,321,313,1.1402,1.0799,64,40,"(70, 100)","(70, 100)",0.1278,0.1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,442374,Swansea City,Fulham,0,0,298,297,2.1309,1.0774,19,87,"(70, 100)","(70, 100)",0.2929,0.0638
376,442375,Tottenham Hotspur,Sunderland,0,0,380,389,1.1211,0.4447,41,121,"(70, 100)","(70, 100)",0.3111,0.1079
377,442376,West Bromwich Albion,Manchester United,0,0,281,273,0.9786,2.0366,50,66,"(70, 100)","(70, 100)",0.2418,0.1779
378,442377,West Ham United,Reading,0,0,332,342,0.9458,0.9240,57,55,"(70, 100)","(70, 100)",0.1608,0.1717


In [48]:
df_dc = pd.read_csv('/Users/luisalawson/Downloads/dc_results.csv')

In [49]:
df_dc['ranking_overall'] = df_dc['dc_overall'].rank(ascending=False)
df_dc['ranking_home'] = df_dc['dc_home'].rank(ascending=False)
df_dc['ranking_away'] = df_dc['dc_away'].rank(ascending=False)

In [50]:
df_dc.head()

,team_name,dc_overall,dc_home,dc_away,dc_home_dif,dc_away_dif,ranking_overall,ranking_home,ranking_away
0,Manchester United,0.716372,0.715204,0.717443,0.001168,-0.001071,5.0,6.0,1.0
1,Southampton,0.718287,0.722656,0.714054,-0.004370,0.004233,3.0,4.0,2.0
2,Liverpool,0.724176,0.739555,0.710407,-0.015379,0.013769,1.0,1.0,3.0
3,Fulham,0.714472,0.719208,0.709649,-0.004736,0.004823,6.0,5.0,4.0
4,Arsenal,0.717665,0.728324,0.707892,-0.010659,0.009773,4.0,3.0,5.0


In [51]:
def get_team_rankings(row, df_dc):
    
    home_team = row['home_team']
    away_team = row['away_team']

    home_team_ranking_overall = df_dc[df_dc['team_name'] == home_team]['ranking_overall'].values
    home_team_ranking_home = df_dc[df_dc['team_name'] == home_team]['ranking_home'].values
    home_team_ranking_away = df_dc[df_dc['team_name'] == home_team]['ranking_away'].values
    home_team_dc_value_home = df_dc[df_dc['team_name'] == home_team]['dc_home'].values
    home_team_dc_value_away= df_dc[df_dc['team_name'] == home_team]['dc_away'].values

    away_team_ranking_overall = df_dc[df_dc['team_name'] == away_team]['ranking_overall'].values
    away_team_ranking_home = df_dc[df_dc['team_name'] == away_team]['ranking_home'].values
    away_team_ranking_away = df_dc[df_dc['team_name'] == away_team]['ranking_away'].values
    away_team_dc_value_home = df_dc[df_dc['team_name'] == away_team]['dc_home'].values
    away_team_dc_value_away= df_dc[df_dc['team_name'] == away_team]['dc_away'].values


    return pd.Series({
        'home_team_ranking_overall': home_team_ranking_overall[0] if len(home_team_ranking_overall) > 0 else None,
        'home_team_ranking_home': home_team_ranking_home[0] if len(home_team_ranking_home) > 0 else None,
        'home_team_ranking_away': home_team_ranking_away[0] if len(home_team_ranking_away) > 0 else None,
        'home_team_dc_value_home': home_team_dc_value_home[0]if len(home_team_dc_value_home) > 0 else None,
        'home_team_dc_value_away': home_team_dc_value_away[0]if len(home_team_dc_value_away) > 0 else None,
        'away_team_ranking_overall': away_team_ranking_overall[0] if len(away_team_ranking_overall) > 0 else None,
        'away_team_ranking_home': away_team_ranking_home[0] if len(away_team_ranking_home) > 0 else None,
        'away_team_ranking_away': away_team_ranking_away[0] if len(away_team_ranking_away) > 0 else None,
        'away_team_dc_value_home': away_team_dc_value_home[0]if len(away_team_dc_value_home) > 0 else None,
        'away_team_dc_value_away': away_team_dc_value_away[0]if len(away_team_dc_value_away) > 0 else None
    })

In [57]:
df_grouped_plays[['home_team_ranking_overall', 'home_team_ranking_home', 'home_team_ranking_away', 'home_team_dc_value_home', 'home_team_dc_value_away', 
                   'away_team_ranking_overall', 'away_team_ranking_home', 'away_team_ranking_away', 'away_team_dc_value_home','away_team_dc_value_away']] = df_grouped_plays.apply(get_team_rankings, axis=1, df_dc=df_dc)

In [58]:
df_grouped_plays

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_faces_dangerous_play,away_faces_dangerous_play,home_team_danger,...,home_team_ranking_home,home_team_ranking_away,home_team_dc_value_home,home_team_dc_value_away,away_team_ranking_overall,away_team_ranking_home,away_team_ranking_away,away_team_dc_value_home,away_team_dc_value_away,correct
0,441999,Arsenal,Sunderland,315,322,2.120635,0.739130,28,96,"(70, 100)",...,3.0,5.0,0.728324,0.707892,18.0,18.0,20.0,0.670015,0.646505,1
1,442000,Everton,Manchester United,350,340,0.651429,1.779412,69,67,"(70, 100)",...,13.0,14.0,0.694702,0.676357,5.0,6.0,1.0,0.715204,0.717443,1
2,442001,Fulham,Norwich City,287,307,1.885017,1.110749,43,66,"(70, 100)",...,5.0,4.0,0.719208,0.709649,19.0,19.0,17.0,0.651262,0.653968,1
3,442002,Manchester City,Southampton,300,278,2.210000,1.226619,53,67,"(70, 100)",...,10.0,10.0,0.701668,0.681605,3.0,4.0,2.0,0.722656,0.714054,0
4,442003,Newcastle United,Tottenham Hotspur,321,313,1.140187,1.079872,64,40,"(70, 100)",...,17.0,19.0,0.670972,0.646586,12.0,11.0,15.0,0.700559,0.673469,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,442374,Swansea City,Fulham,298,297,2.130872,1.077441,19,87,"(70, 100)",...,15.0,12.0,0.683717,0.677075,6.0,5.0,4.0,0.719208,0.709649,0
376,442375,Tottenham Hotspur,Sunderland,380,389,1.121053,0.444730,41,121,"(70, 100)",...,11.0,15.0,0.700559,0.673469,18.0,18.0,20.0,0.670015,0.646505,1
377,442376,West Bromwich Albion,Manchester United,281,273,0.978648,2.036630,50,66,"(70, 100)",...,12.0,11.0,0.696600,0.681150,5.0,6.0,1.0,0.715204,0.717443,1
378,442377,West Ham United,Reading,332,342,0.945783,0.923977,57,55,"(70, 100)",...,14.0,13.0,0.689212,0.676688,20.0,20.0,18.0,0.648819,0.649343,1


In [53]:
condition1 = (df_grouped_plays['home_passes_per_play'] > df_grouped_plays['away_passes_per_play'])
condition2 = (df_grouped_plays['home_team_ranking_overall'] < df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_dc_value_home'] > df_grouped_plays['away_team_dc_value_away'])

df_grouped_plays['correct'] = np.where(condition1 & condition2, 1, 0)

condition3 = (df_grouped_plays['home_passes_per_play'] < df_grouped_plays['away_passes_per_play'])
condition4 = (df_grouped_plays['home_team_ranking_overall'] > df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_dc_value_home'] < df_grouped_plays['away_team_dc_value_away'])

df_grouped_plays['correct'] = np.where(condition3 & condition4, 1, df_grouped_plays['correct'])

In [54]:
excel_file_path = '/Users/luisalawson/Downloads/pass_dc_comparison.xlsx'  

df_grouped_plays.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /Users/luisalawson/Downloads/pass_dc_comparison.xlsx


In [59]:
df_grouped_plays[(df_grouped_plays['home_team']=='Manchester United') | (df_grouped_plays['away_team']=='Manchester United')]

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_faces_dangerous_play,away_faces_dangerous_play,home_team_danger,...,home_team_ranking_home,home_team_ranking_away,home_team_dc_value_home,home_team_dc_value_away,away_team_ranking_overall,away_team_ranking_home,away_team_ranking_away,away_team_dc_value_home,away_team_dc_value_away,correct
1,442000,Everton,Manchester United,350,340,0.651429,1.779412,69,67,"(70, 100)",...,13.0,14.0,0.694702,0.676357,5.0,6.0,1.0,0.715204,0.717443,1
13,442012,Manchester United,Fulham,303,297,1.983498,1.309764,50,95,"(70, 100)",...,6.0,1.0,0.715204,0.717443,6.0,5.0,4.0,0.719208,0.709649,1
24,442023,Southampton,Manchester United,301,295,1.441860,1.861017,55,32,"(70, 100)",...,4.0,2.0,0.722656,0.714054,5.0,6.0,1.0,0.715204,0.717443,0
34,442033,Manchester United,Wigan Athletic,297,305,1.370370,1.163934,44,80,"(70, 100)",...,6.0,1.0,0.715204,0.717443,7.0,9.0,6.0,0.702730,0.706317,1
41,442040,Liverpool,Manchester United,328,328,1.301829,1.192073,35,79,"(70, 100)",...,1.0,3.0,0.739555,0.710407,5.0,6.0,1.0,0.715204,0.717443,1
54,442053,Manchester United,Tottenham Hotspur,327,321,2.012232,0.654206,39,85,"(70, 100)",...,6.0,1.0,0.715204,0.717443,12.0,11.0,15.0,0.700559,0.673469,1
63,442062,Newcastle United,Manchester United,347,335,1.178674,1.432836,40,67,"(70, 100)",...,17.0,19.0,0.670972,0.646586,5.0,6.0,1.0,0.715204,0.717443,1
72,442071,Manchester United,Stoke City,278,282,2.161871,1.000000,51,41,"(70, 100)",...,6.0,1.0,0.715204,0.717443,8.0,8.0,8.0,0.704946,0.691324,1
82,442081,Chelsea,Manchester United,325,322,1.193846,1.313665,47,99,"(70, 100)",...,2.0,7.0,0.739376,0.698745,5.0,6.0,1.0,0.715204,0.717443,0
92,442091,Manchester United,Arsenal,300,318,1.610000,1.732704,38,61,"(70, 100)",...,6.0,1.0,0.715204,0.717443,4.0,3.0,5.0,0.728324,0.707892,1


### Rival posession in dangerous area

In [27]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_csv('/Users/pgule/Documents/Sample Data/epl.csv', delimiter=',')

# Filtrar por el partido específico y el equipo contrario (Sunderland vs Arsenal)
match_id = 441999
arsenal_team_id = 3  # Supongamos que el id del Arsenal es 3
df_match = df[df['match_id'] == match_id]

# Filtrar las jugadas del equipo rival (Sunderland)
rival_team_id = df_match[df_match['team_id'] != arsenal_team_id]['team_id'].unique()[0]
df_rival = df_match[df_match['team_id'] == rival_team_id]

# Ordenar las jugadas por tiempo para asegurar la secuencia correcta
df_rival = df_rival.sort_values(by=['min', 'sec'])

# Inicializar variables
df_rival['possession_id'] = 0
current_possession = 1

# Identificar el inicio y fin de posesiones del equipo rival
for i, row in df_rival.iterrows():
    if row['description'] in ['Interception', 'Ball recovery', 'Tackle', 'Goalkeeper Save', 'Out', 'Foul', 'Clearance', 'Pass'] and row['outcome'] == 0:
        current_possession += 1
    df_rival.at[i, 'possession_id'] = current_possession

# Identificar las posesiones que pasaron por áreas peligrosas
df_rival['in_dangerous_area'] = df_rival['x'].apply(lambda x: 70 <= x <= 100)

# Agrupar por posesión para identificar posesiones peligrosas
dangerous_plays = df_rival.groupby('possession_id')['in_dangerous_area'].max().reset_index()

# Calcular RPDA/RP para el rival de Arsenal
total_possessions = df_rival['possession_id'].nunique()
dangerous_possessions = dangerous_plays[dangerous_plays['in_dangerous_area'] == True]['possession_id'].nunique()

# Calcular el ratio RPDA/RP
rpda_rp_ratio = dangerous_possessions / total_possessions if total_possessions > 0 else 0

print(f"RPDA/RP ratio para el rival de Arsenal: {rpda_rp_ratio}")


TypeError: cannot convert the series to <class 'float'>

In [21]:
#quiero ver df_match en excel, commo lo exporto?
excel_file_path = '/Users/pgule/Documents/Sample Data/RPDA_Prueba1.xlsx'

df_rival.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")


DataFrame saved to /Users/pgule/Documents/Sample Data/RPDA_Prueba1.xlsx
